# Build mesh and lead field

A guided, CPU-only walkthrough that loads the CRT demo mesh, ingests a Purkinje UV tree, seeds PMJs, computes the lead field, and produces a PyVista screenshot.

**Prerequisites**

- Repository data bundle available at `data/crtdemo`
- CPU runtime (no GPU required)
- PyVista can run off-screen; screenshots are saved to the `examples/` directory


In [ ]:
from pathlib import Path
import numpy as np
import pyvista as pv
from IPython.display import Image, display

from examples.build_mesh_and_lead_field import (
    DEFAULT_DATA_DIR,
    DEFAULT_TREE_PATH,
    DEFAULT_SCREENSHOT,
    load_uv_tree,
    map_pmjs_to_myocardium,
    plot_activation,
)
from myocardial_mesh import MyocardialMesh


## 1. Load the sample myocardial mesh

In [ ]:
data_dir = DEFAULT_DATA_DIR
myo = MyocardialMesh(
    myo_mesh=str(data_dir / 'crtdemo_mesh_oriented.vtk'),
    electrodes_position=str(data_dir / 'electrode_pos.pkl'),
    fibers=str(data_dir / 'crtdemo_f0_oriented.vtk'),
    device='cpu',
)
myo


## 2. Import a small Purkinje UV tree

The bundled LV tree is a VTK export from the UV toolchain. Leaf nodes are treated as Purkinje–myocardium junctions (PMJs) for this quickstart.

In [ ]:
tree = load_uv_tree(DEFAULT_TREE_PATH)
tree.xyz.shape, tree.edges.shape, tree.pmj_idx.size


## 3. Embed the tree with the helper

Snap a PMJ subset to the closest myocardial nodes to create activation seeds.

In [ ]:
rng = np.random.default_rng(0)
sample_size = 256
pmj_idx = (
    rng.choice(tree.pmj_idx, size=sample_size, replace=False)
    if sample_size < tree.pmj_idx.size
    else tree.pmj_idx
)
pmj_xyz = tree.xyz[pmj_idx]
pmj_nodes, pmj_nodes_xyz = map_pmjs_to_myocardium(pmj_xyz, myocardium=myo)
len(pmj_nodes)


## 4. Compute the lead field (CPU)

Run the FIM solver from the embedded PMJs, then synthesise a 12-lead ECG from the activation field.

In [ ]:
activation = myo.activate_fim(
    x0=pmj_nodes_xyz,
    x0_vals=np.zeros(pmj_nodes_xyz.shape[0], dtype=float),
    return_only_pmjs=False,
)
lead_field = myo.get_lead_field()
ecg = myo.new_get_ecg(record_array=True)
(float(np.min(activation[np.isfinite(activation)])),
 float(np.max(activation[np.isfinite(activation)])),
 len(lead_field),
 ecg.dtype.names[:5],
)


## 5. Visualise the potentials (PyVista screenshot)

In [ ]:
pv.start_xvfb()
shot_path = DEFAULT_SCREENSHOT.with_name('notebook_build_mesh_and_lead_field.png')
shot_path.parent.mkdir(parents=True, exist_ok=True)
plot_activation(myo, pmj_nodes, shot_path)
display(Image(filename=str(shot_path)))


_Note: If PyVista cannot render off-screen in your environment, the helper will fall back to a Matplotlib snapshot while still attempting PyVista first._